In [1]:
cd ~/SageMaker/house-xgb

/home/ec2-user/SageMaker/house-xgb


In [2]:
ls

Dockerfile  train.csv  train.py


In [3]:
pwd

'/home/ec2-user/SageMaker/house-xgb'

In [4]:
import os
for f in os.listdir():
    print(f)

Dockerfile
train.py
.ipynb_checkpoints


In [40]:
! docker build -t house-xgb .

Sending build context to Docker daemon  472.1kB
Step 1/7 : FROM python:3.10
 ---> eeec5526d75f
Step 2/7 : RUN pip3 install -U scikit-learn
 ---> Using cache
 ---> 95358409a497
Step 3/7 : RUN pip3 install xgboost
 ---> Using cache
 ---> cf2245f8857b
Step 4/7 : RUN pip3 install pandas
 ---> Using cache
 ---> 6dc4b56e335a
Step 5/7 : RUN pip3 install sagemaker-training
 ---> Using cache
 ---> 660416a70b5d
Step 6/7 : COPY train.py /opt/ml/code/train.py
 ---> f3629c5431dd
Step 7/7 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 30dfaaad0997
Removing intermediate container 30dfaaad0997
 ---> 2dee389f192a
Successfully built 2dee389f192a
Successfully tagged house-xgb:latest


In [41]:
!docker images

REPOSITORY   TAG       IMAGE ID       CREATED          SIZE
house-xgb    latest    2dee389f192a   5 seconds ago    2.25GB
<none>       <none>    97341442eb2d   4 minutes ago    2.25GB
<none>       <none>    1497a6da552e   12 minutes ago   2.25GB
<none>       <none>    2a5ac49c2293   3 hours ago      2.25GB
<none>       <none>    aa98e60c3a2f   3 hours ago      2.25GB
python       3.10      eeec5526d75f   7 weeks ago      1GB


In [46]:
# import boto3
# s3 = boto3.client('s3')
# with open('train.csv', 'wb') as f:
#     s3.download_fileobj('houses-xgb-01', 'data/train.csv', f)

In [45]:
# train_file = 'train.csv'
# s3_bucket = 'houses-xgb-01'
# s3 = boto3.client('s3')
# with open(train_file, 'wb') as f:
#     s3.download_fileobj(s3_bucket, train_file, f)

In [44]:
# ls

In [43]:
# import pandas as pd
# import csv
# list = []
# with open('train.csv', mode ='r') as file:
#   csvFile = csv.reader(file)
#   for lines in csvFile:
#         list = lines
#         print(lines)
#         break

In [20]:
# Test the container locally

In [42]:
import sagemaker
from sagemaker.estimator import Estimator

xgb_model = Estimator(image_uri='house-xgb',
                      role=sagemaker.get_execution_role(),
                      instance_count=1,
                      instance_type='local')

xgb_model.fit()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: house-xgb-2024-05-13-04-08-24-576
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-lgcd9:
    command: train
    container_name: 4h2g3cl1yp-algo-1-lgcd9
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: house-xgb
    networks:
      sagemaker-local:
        aliases:
        - algo-1-lgcd9
    stdi

 Container 4h2g3cl1yp-algo-1-lgcd9  Creating
 Container 4h2g3cl1yp-algo-1-lgcd9  Created
Attaching to 4h2g3cl1yp-algo-1-lgcd9
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:25,966 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:25,967 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:25,981 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:25,988 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:25,989 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:26,000 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
4h2g3cl1yp-algo-1-lgcd9  | 2024-05-13 04:08:26

INFO:root:creating /tmp/tmpgf55b3fi/artifacts/output/data
INFO:root:copying /tmp/tmpgf55b3fi/algo-1-lgcd9/output/success -> /tmp/tmpgf55b3fi/artifacts/output
INFO:root:copying /tmp/tmpgf55b3fi/model/model.ubj -> /tmp/tmpgf55b3fi/artifacts/model


4h2g3cl1yp-algo-1-lgcd9 exited with code 0
Aborting on container exit...
 Container 4h2g3cl1yp-algo-1-lgcd9  Stopping
 Container 4h2g3cl1yp-algo-1-lgcd9  Stopped


INFO:sagemaker.local.image:===== Job Complete =====


In [47]:
#Push the container to Amazon Elastic Container Registry (Amazon ECR)

In [48]:
%%sh

# Specify an Docker image name
image_name=house-xgb

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${image_name} .
docker tag ${image_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  10.75kB
Step 1/7 : FROM python:3.10
 ---> eeec5526d75f
Step 2/7 : RUN pip3 install -U scikit-learn
 ---> Using cache
 ---> 95358409a497
Step 3/7 : RUN pip3 install xgboost
 ---> Using cache
 ---> cf2245f8857b
Step 4/7 : RUN pip3 install pandas
 ---> Using cache
 ---> 6dc4b56e335a
Step 5/7 : RUN pip3 install sagemaker-training
 ---> Using cache
 ---> 660416a70b5d
Step 6/7 : COPY train.py /opt/ml/code/train.py
 ---> Using cache
 ---> f3629c5431dd
Step 7/7 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> 2dee389f192a
Successfully built 2dee389f192a
Successfully tagged house-xgb:latest
The push refers to repository [047922237497.dkr.ecr.us-east-1.amazonaws.com/house-xgb]
3cd0b17eaa0a: Preparing
475992d08836: Preparing
278cad93e8ef: Preparing
c1275a42b05c: Preparing
d7da6810caef: Preparing
9a6c88132041: Preparing
82553f50c177: Preparing
2c746e256285: Preparing
89ca33c95b2e: Preparing
83db175c22e2: Preparing
c5d13b2949a2: Prepar

In [49]:
# Get the Amazon ECR image of your training container.

In [50]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'house-xgb'
tag = ':latest'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'

byoc_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

byoc_image_uri


'047922237497.dkr.ecr.us-east-1.amazonaws.com/house-xgb:latest'

In [51]:
# Use the ecr_image retrieved from the previous step to configure a SageMaker estimator object.

In [55]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

output_location = 's3://houses-xgb-01'

estimator = Estimator(image_uri=byoc_image_uri,
                      role=get_execution_role(),
                      base_job_name='houses-xgb',
                      instance_count=1,
                      instance_type='ml.g4dn.xlarge',
                      output_path=output_location)

#train your model
estimator.fit()


INFO:sagemaker:Creating training-job with name: houses-xgb-2024-05-13-05-07-19-443


2024-05-13 05:07:19 Starting - Starting the training job...
2024-05-13 05:07:36 Starting - Preparing the instances for training...
2024-05-13 05:08:17 Downloading - Downloading the training image...
2024-05-13 05:08:48 Training - Training image download completed. Training in progress.
2024-05-13 05:08:48 Uploading - Uploading generated training model2024-05-13 05:08:40,156 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-05-13 05:08:40,186 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-05-13 05:08:40,214 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-05-13 05:08:40,225 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {},
    "current_host": "algo-1",
    "current_instance_group": "homogeneousCluster",
    "current_instance_group_hosts": [
        "algo-1"
 